# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 35

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 10'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
340,Lloris,Tottenham,53,18.043478,83,80.645967,4.6,4.469535,4.251266,4.493724,1.941442
411,Patrício,Wolves,53,34.878049,143,152.537547,4.1,4.373454,3.817550,4.051800,1.899707
471,Henderson,Sheffield Utd,52,32.978723,155,139.288386,4.7,4.223583,4.739600,4.541944,1.852164
93,Pope,Burnley,50,34.782609,160,143.393849,4.6,4.122573,4.168773,4.231405,1.843671
168,Schmeichel,Leicester City,55,35.238095,148,142.994669,4.2,4.057957,3.795290,4.675402,1.730319
235,de Gea,Manchester Utd,53,35.000000,126,135.605004,3.6,3.874429,3.392440,3.881571,1.682944
131,Guaita,Crystal Palace,49,31.951220,131,123.337223,4.1,3.860173,2.839173,3.678243,1.743848
189,Alisson,Liverpool,62,26.222222,118,100.728994,4.5,3.841360,3.490746,3.609593,1.542727
14,Leno,Arsenal,48,30.000000,114,114.594261,3.8,3.819809,NaN,4.581548,1.743496


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
182,Alexander-Arnold,Liverpool,78,35.000000,196,181.186842,5.6,5.176767,2.956475,3.641013,1.853580
401,Doherty,Wolves,65,32.826087,151,156.382906,4.6,4.763983,4.376757,5.172229,1.868588
181,Robertson,Liverpool,70,33.000000,165,151.529147,5.0,4.591792,3.459070,3.391348,1.735534
103,Alonso,Chelsea,61,14.909091,82,68.093969,5.5,4.567278,2.216987,3.747535,1.849237
105,Azpilicueta,Chelsea,61,33.142857,116,146.388427,3.5,4.416892,3.628976,4.421621,1.788347
297,Lundstram,Sheffield Utd,46,30.681818,135,131.637243,4.4,4.290399,1.950203,2.972689,2.000409
183,van Dijk,Liverpool,65,34.791667,167,143.941355,4.8,4.137237,3.099781,3.387201,1.622758
159,Pereira,Leicester City,62,27.954545,123,111.928060,4.4,4.003931,NaN,4.223640,1.608017
122,Wan-Bissaka,Manchester Utd,55,31.666667,114,125.920055,3.6,3.976423,3.280610,4.022225,1.695552


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,126,30.821918,225,212.132620,7.3,6.882525,7.575788,6.729770,1.938932
618,Fernandes,Manchester Utd,90,11.000000,99,72.807329,9.0,6.618848,7.112575,7.059028,2.206283
215,De Bruyne,Manchester City,108,33.235294,226,206.767321,6.8,6.221318,6.184559,6.463953,1.893087
214,Sterling,Manchester City,118,30.169492,178,177.137379,5.9,5.871408,7.780645,7.253246,1.709233
192,Mané,Liverpool,122,32.063492,202,182.596669,6.3,5.694847,4.037132,4.438880,1.630430
342,Son,Tottenham,96,26.842105,153,136.343558,5.7,5.079466,5.647115,5.253791,1.639391
239,Martial,Manchester Utd,84,29.193548,181,147.533509,6.2,5.053634,7.237525,6.583081,1.743669
217,Mahrez,Manchester City,86,29.818182,164,146.037153,5.5,4.897587,6.080096,5.528239,1.670063
389,Antonio,West Ham,70,21.111111,95,102.600575,4.5,4.860027,9.745933,7.816765,1.836918


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
233,Rashford,Manchester Utd,90,27.931034,162,157.595507,5.8,5.642308,5.395422,5.358386,1.880769
166,Vardy,Leicester City,98,32.063492,202,178.430435,6.3,5.564910,7.670713,6.387197,1.777645
409,Jiménez,Wolves,81,34.901961,178,173.510435,5.1,4.971366,4.390713,4.466812,1.746760
11,Aubameyang,Arsenal,109,33.214286,186,163.640290,5.6,4.926804,5.395686,5.267403,1.492286
210,Agüero,Manchester City,117,24.000000,132,117.090435,5.5,4.878768,NaN,4.342392,1.426321
187,Firmino,Liverpool,93,35.250000,141,161.440580,4.0,4.579875,3.283173,3.207870,1.501800
460,Abraham,Chelsea,74,31.063830,146,141.730435,4.7,4.562555,3.236286,3.341049,1.677229
313,Ings,Southampton,75,35.098039,179,159.310145,5.1,4.539004,4.692460,4.509962,1.657410
211,Jesus,Manchester City,98,31.219512,128,137.660580,4.1,4.409440,7.954095,5.740430,1.408544


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,126,30.821918,225,212.132620,7.3,6.882525,7.575788,6.729770,1.938932
618,Fernandes,Manchester Utd,90,11.000000,99,72.807329,9.0,6.618848,7.112575,7.059028,2.206283
215,De Bruyne,Manchester City,108,33.235294,226,206.767321,6.8,6.221318,6.184559,6.463953,1.893087
214,Sterling,Manchester City,118,30.169492,178,177.137379,5.9,5.871408,7.780645,7.253246,1.709233
192,Mané,Liverpool,122,32.063492,202,182.596669,6.3,5.694847,4.037132,4.438880,1.630430
233,Rashford,Manchester Utd,90,27.931034,162,157.595507,5.8,5.642308,5.395422,5.358386,1.880769
166,Vardy,Leicester City,98,32.063492,202,178.430435,6.3,5.564910,7.670713,6.387197,1.777645
182,Alexander-Arnold,Liverpool,78,35.000000,196,181.186842,5.6,5.176767,2.956475,3.641013,1.853580
342,Son,Tottenham,96,26.842105,153,136.343558,5.7,5.079466,5.647115,5.253791,1.639391


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
389,Antonio,West Ham,70,21.111111,95,102.600575,4.5,4.860027,9.745933,7.816765,1.836918
214,Sterling,Manchester City,118,30.169492,178,177.137379,5.9,5.871408,7.780645,7.253246,1.709233
618,Fernandes,Manchester Utd,90,11.000000,99,72.807329,9.0,6.618848,7.112575,7.059028,2.206283
191,Salah,Liverpool,126,30.821918,225,212.132620,7.3,6.882525,7.575788,6.729770,1.938932
113,Willian,Chelsea,74,34.255319,161,152.128283,4.7,4.441012,7.100021,6.631412,1.632548
239,Martial,Manchester Utd,84,29.193548,181,147.533509,6.2,5.053634,7.237525,6.583081,1.743669
215,De Bruyne,Manchester City,108,33.235294,226,206.767321,6.8,6.221318,6.184559,6.463953,1.893087
166,Vardy,Leicester City,98,32.063492,202,178.430435,6.3,5.564910,7.670713,6.387197,1.777645
211,Jesus,Manchester City,98,31.219512,128,137.660580,4.1,4.409440,7.954095,5.740430,1.408544


In [11]:
metric = 'valuePoints metric'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,90,11.000000,99,72.807329,9.0,6.618848,7.112575,7.059028,2.206283
297,Lundstram,Sheffield Utd,46,30.681818,135,131.637243,4.4,4.290399,1.950203,2.972689,2.000409
340,Lloris,Tottenham,53,18.043478,83,80.645967,4.6,4.469535,4.251266,4.493724,1.941442
191,Salah,Liverpool,126,30.821918,225,212.132620,7.3,6.882525,7.575788,6.729770,1.938932
411,Patrício,Wolves,53,34.878049,143,152.537547,4.1,4.373454,3.817550,4.051800,1.899707
215,De Bruyne,Manchester City,108,33.235294,226,206.767321,6.8,6.221318,6.184559,6.463953,1.893087
233,Rashford,Manchester Utd,90,27.931034,162,157.595507,5.8,5.642308,5.395422,5.358386,1.880769
341,Gazzaniga,Tottenham,41,18.125000,58,68.925692,3.2,3.802797,NaN,1.653034,1.878067
401,Doherty,Wolves,65,32.826087,151,156.382906,4.6,4.763983,4.376757,5.172229,1.868588
